In [1]:
# import statements

import requests
import pandas as pd
import ipywidgets as widgets

In [2]:
# This code writes the function for the API POST request for the fullreport estimates
# this function will accept a FIADB-API fullreport URL and return dataframes for the estimates as well as subtotals, and totals where available.

def fiadb_api_POST(parameterDictionary):
    # make request
    resp = requests.post(r"https://apps.fs.usda.gov/fiadb-api/fullreport",data=parameterDictionary)
    # parse response to json
    data = resp.json()

    # create output dictionary and populate it with estimate data frames
    outDict = {}
    # append estimates
    outDict['estimates'] = pd.DataFrame(data['estimates'])

    # append subtotals and totals if present
    if 'subtotals' in data.keys():
        subT = {}
        for i in data['subtotals'].keys():
            subT[i] = pd.DataFrame(data['subtotals'][i])
        outDict['subtotals'] = subT
        outDict['totals'] = pd.DataFrame(data['totals'])

    # append metadata
    outDict['metadata'] = data['metadata']
    return outDict

In [3]:
# This code writes the furnction for the API GET request to retreive all available Evaluation groups (wc codes)
# The EVALID column consists of the state FIPS code and the 4 digit inventory code
# The EVAL_GRP column is the 6 digit code that we need to send to the API in our POST request for a given state and report year, as the wc parameter in our POST dictionary

def fiadb_api_wc_GET(url):
    # make request
    resp = requests.get(url)
    # parse response to json
    data = resp.json()

    output = pd.DataFrame(data)

    return output

url = 'https://apps.fs.usda.gov/fiadb-api/fullreport/parameters/wc?outputFormat=JSON'
wc_data = fiadb_api_wc_GET(url)

In [32]:
# This code block will use the wc data from our GET request to populate a dropdown list

wc_data_sort = list(wc_data[['EVALID']].EVALID)

stateEVALID = widgets.Dropdown(
    options=wc_data_sort,
    value=wc_data_sort[0],
    description='Choose State and Report Year EVALID:',
    disabled=False,
)

display(stateEVALID)

# The user should select which state/report they would like to pull data for using the dropdown below:
# THE LAST 4 DIGITS OF THE NUMBER IS THE YEAR OF THE REPORT

Dropdown(description='Choose State and Report Year EVALID:', options=(' 012022 ALABAMA', ' 012021 ALABAMA', ' …

In [17]:
# Now we need to split the resulting string variable from the dropdown choice into the report code and state name:

statereportlist = stateEVALID.value.split()
statename = statereportlist[1]
statereportcode = statereportlist[0]

print('Your Selected State is: ' + statename)
print('EVALID CODE: ' + statereportcode)

Your Selected State is: COLORADO
EVALID CODE: 082019


In [18]:
# The following code blocks will make the API requests for each estimate type

# 79_SAMPLEDLANDWATER_ACRES
slw_parameters = {'snum':'79', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
sampledlandwater_acres_GET = fiadb_api_POST(slw_parameters)

In [19]:
# 2_FORESTLAND_ACRES
fl_parameters = {'snum':'2', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
forestland_acres_GET = fiadb_api_POST(fl_parameters)

In [20]:
# 20_NETSAWLOGVOLSAWTMBR_BRDFT
nslv_parameters = {'snum':'20', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
nslv_brdft_GET = fiadb_api_POST(nslv_parameters)

In [21]:
# 14_NETMERCHBOLELIVETREES_CBFT
nmblt_parameters = {'snum':'14', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
nmblt_cbft_GET = fiadb_api_POST(nmblt_parameters)

In [22]:
# 203_AVGNETGROWTHSAWTMBR_BRDFT
angst_parameters = {'snum':'203', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
angst_brdft_GET = fiadb_api_POST(angst_parameters)

In [23]:
# Average annual gross growth of sawlog volume of sawtimber trees, in board feet
# 1203_AVGANNGROSSGROWTHSAWTMBR_BRDFT
aaggst_parameters = {'snum':'1203', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
aaggst_brdft_GET = fiadb_api_POST(aaggst_parameters)

In [24]:
# 227_AVGREMOVALSAWTMBR_BRDFT
arst_parameters = {'snum':'227', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
arst_brdft_GET = fiadb_api_POST(arst_parameters)

In [25]:
# 98_CARBONLIVEABOVEGRND_MT
clab_parameters = {'snum':'98', 'wc':statereportcode, 'rselected':'County code and name', 'cselected':'State', 'outputFormat':'NJSON'}
clab_mt_GET = fiadb_api_POST(clab_parameters)

In [26]:
# This code block will format the returned data for each estimate type, and rename the columns with the Estimate ID at the beginning

# 79_SAMPLEDLANDWATER_ACRES
sampledlandwater_acres = sampledlandwater_acres_GET['estimates']
slw_acres = sampledlandwater_acres[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
slw_79_acres = slw_acres.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '79_SAMPLEDLANDWATER_ACRES', 'SE': '79_SE', 'SE_PERCENT': '79_SE_PERCENT', 'PLOT_COUNT': '79_PLOT_COUNT'})

# 2_FORESTLAND_ACRES
forestland_acres = forestland_acres_GET['estimates']
fl_acres = forestland_acres[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
fl_2_acres = fl_acres.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '2_FORESTLAND_ACRES', 'SE': '2_SE', 'SE_PERCENT': '2_SE_PERCENT', 'PLOT_COUNT': '2_PLOT_COUNT'})

# 20_NETSAWLOGVOLSAWTMBR_BRDFT
nslv_brdft = nslv_brdft_GET['estimates']
nslv_brdft_temp = nslv_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
nslv_20_brdft = nslv_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '20_NETSAWLOGVOLSAWTMBR_BRDFT', 'SE': '20_SE', 'SE_PERCENT': '20_SE_PERCENT', 'PLOT_COUNT': '20_PLOT_COUNT'})

# 14_NETMERCHBOLELIVETREES_CBFT
nmblt_cbft = nmblt_cbft_GET['estimates']
nmblt_cbft_temp = nmblt_cbft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
nmblt_14_cbft = nmblt_cbft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '14_NETMERCHBOLELIVETREES_CBFT', 'SE': '14_SE', 'SE_PERCENT': '14_SE_PERCENT', 'PLOT_COUNT': '14_PLOT_COUNT'})

# 203_AVGNETGROWTHSAWTMBR_BRDFT
angst_brdft = angst_brdft_GET['estimates']
angst_brdft_temp = angst_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
angst_203_brdft = angst_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '203_AVGNETGROWTHSAWTMBR_BRDFT', 'SE': '203_SE', 'SE_PERCENT': '203_SE_PERCENT', 'PLOT_COUNT': '203_PLOT_COUNT'})

# 1203_AVGANNGROSSGROWTHSAWTMBR_BRDFT
aaggst_brdft = aaggst_brdft_GET['estimates']
aaggst_brdft_temp = aaggst_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
aaggst_1203_brdft = aaggst_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '1203_AVGANNGROSSGROWTHSAWTMBR_BRDFT', 'SE': '1203_SE', 'SE_PERCENT': '1203_SE_PERCENT', 'PLOT_COUNT': '1203_PLOT_COUNT'})

# 227_AVGREMOVALSAWTMBR_BRDFT
arst_brdft = arst_brdft_GET['estimates']
arst_brdft_temp = arst_brdft[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
arst_227_brdft = arst_brdft_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '227_AVGREMOVALSAWTMBR_BRDFT', 'SE': '227_SE', 'SE_PERCENT': '227_SE_PERCENT', 'PLOT_COUNT': '227_PLOT_COUNT'})

# 98_CARBONLIVEABOVEGRND_MT
clab_mt = clab_mt_GET['estimates']
clab_mt_temp = clab_mt[['GRP1', 'ESTIMATE', 'SE', 'SE_PERCENT', 'PLOT_COUNT']]
clab_98_mt = clab_mt_temp.rename(columns={'GRP1': 'STATE_COUNTY_CODE', 'ESTIMATE': '98_CARBONLIVEABOVEGRND_MT', 'SE': '98_SE', 'SE_PERCENT': '98_SE_PERCENT', 'PLOT_COUNT': '98_PLOT_COUNT'})

In [27]:
# This code block performs the Left Join on dataframes
state_data = slw_79_acres.merge(
    fl_2_acres, how='left', on='STATE_COUNTY_CODE').merge(
    nslv_20_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    nmblt_14_cbft, how='left', on='STATE_COUNTY_CODE').merge(
    angst_203_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    aaggst_1203_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    arst_227_brdft, how='left', on='STATE_COUNTY_CODE').merge(
    clab_98_mt, how='left', on='STATE_COUNTY_CODE')

# Rounding the data to 2 decimal places
state_data = state_data.round(2)

# Filling na values as zeros
state_data_final = state_data.fillna('')

In [28]:
#This code block exports the results as a .csv named with the state and fia report code

state_data_final.to_csv('./state_data_csv_files/' + statename + '_' + statereportcode + '.csv', index=False)